In [2]:
def crop_right(img):
    # draw filled rectangle in white on black background as mask
    mask = np.zeros_like(img)
    mask = cv2.rectangle(mask,(240,300),(1186,700),(255,255,255),-1)
    # apply mask to image
    result = cv2.bitwise_and(img, mask)
    return result

def crop_left(img):
    # draw filled rectangle in white on black background as mask
    mask = np.zeros_like(img)
    mask = cv2.rectangle(mask,(360,230),(1273,684),(255,255,255),-1)
    # apply mask to image
    result = cv2.bitwise_and(img, mask)
    return result


In [3]:
from skimage.metrics import structural_similarity
import cv2
import numpy as np

left_first = cv2.imread('../../rectified_images/left/0.png')
left_first_cropped = crop_left(left_first)

after = cv2.imread('../../rectified_images/left/1068.png')
after_cropped = crop_left(after)



# Convert images to grayscale
before_gray = cv2.cvtColor(left_first_cropped, cv2.COLOR_BGR2GRAY)
after_gray = cv2.cvtColor(after_cropped, cv2.COLOR_BGR2GRAY)

before_gray = cv2.blur(before_gray, (10,10))
after_gray = cv2.blur(after_gray, (10,10))
# Compute SSIM between two images
(score, diff) = structural_similarity(before_gray, after_gray, full=True)
print("Image similarity", score)

# The diff image contains the actual image differences between the two images
# and is represented as a floating point data type in the range [0,1]
# so we must convert the array to 8-bit unsigned integers in the range
# [0,255] before we can use it with OpenCV
diff = (diff * 255).astype("uint8")

# Threshold the difference image, followed by finding contours to
# obtain the regions of the two input images that differ
thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

mask = np.zeros(left_first_cropped.shape, dtype='uint8')
filled_after = after_cropped.copy()

for c in contours:
    area = cv2.contourArea(c)
    if area > 2000:
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(left_first_cropped, (x, y), (x + w, y + h), (36,255,12), 2)
        cv2.rectangle(after_cropped, (x, y), (x + w, y + h), (36,255,12), 2)
        cv2.drawContours(mask, [c], 0, (0,255,0), -1)
        cv2.drawContours(filled_after, [c], 0, (0,255,0), -1)

cv2.imshow('before', left_first_cropped)
cv2.imshow('after', after_cropped)
cv2.imshow('diff',diff)
cv2.imshow('mask',mask)
cv2.imshow('filled after',filled_after)
cv2.waitKey(0)

Image similarity 0.980220162689235


189

In [7]:
# importing the module
import cv2

# function to display the coordinates of
# of the points clicked on the image
def click_event(event, x, y, flags, params):

	# checking for left mouse clicks
	if event == cv2.EVENT_LBUTTONDOWN:

		# displaying the coordinates
		# on the Shell
		print(x, ' ', y)

		# displaying the coordinates
		# on the image window
		font = cv2.FONT_HERSHEY_SIMPLEX
		cv2.putText(img, str(x) + ',' +
					str(y), (x,y), font,
					1, (255, 0, 0), 2)
		cv2.imshow('image', img)

	# checking for right mouse clicks	
	if event==cv2.EVENT_RBUTTONDOWN:

		# displaying the coordinates
		# on the Shell
		print(x, ' ', y)

		# displaying the coordinates
		# on the image window
		font = cv2.FONT_HERSHEY_SIMPLEX
		b = img[y, x, 0]
		g = img[y, x, 1]
		r = img[y, x, 2]
		cv2.putText(img, str(b) + ',' +
					str(g) + ',' + str(r),
					(x,y), font, 1,
					(255, 255, 0), 2)
		cv2.imshow('image', img)

# driver function
if __name__=="__main__":

	# reading the image
	img = cv2.imread('../../rectified_images/right/0.png', 1)

	# displaying the image
	cv2.imshow('image', img)

	# setting mouse handler for the image
	# and calling the click_event() function
	cv2.setMouseCallback('image', click_event)

	# wait for a key to be pressed to exit
	cv2.waitKey(0)

	# close the window
	cv2.destroyAllWindows()


224   326
1191   593


In [5]:
def crop_rect(img, rect):
    # get the parameter of the small rectangle
    center, size, angle = rect[0], rect[1], rect[2]
    center, size = tuple(map(int, center)), tuple(map(int, size))

    # get row and col num in img
    height, width = img.shape[0], img.shape[1]

    # calculate the rotation matrix
    M = cv2.getRotationMatrix2D(center, angle, 1)
    # rotate the original image
    img_rot = cv2.warpAffine(img, M, (width, height))

    # now rotated rectangle becomes vertical, and we crop it
    img_crop = cv2.getRectSubPix(img_rot, size, center)

    return img_crop, img_rot

In [13]:
import cv2
import numpy as np

# read image
img = cv2.imread('../../rectified_images/right/0.png')

# draw filled circle in white on black background as mask
mask = np.zeros_like(img)
#for left first image
#mask = cv2.rectangle(mask,(360,290),(1273,684),(255,255,255),-1)
#for right first image
mask = cv2.rectangle(mask,(240,300),(1186,700),(255,255,255),-1)
# apply mask to image
result = cv2.bitwise_and(img, mask)

# save results
cv2.imwrite('jeep_mask.png', mask)
cv2.imwrite('jeep_masked.png', result)

cv2.imshow('masked image', result)
cv2.waitKey(0)
cv2.destroyAllWindows()